In [3]:
import openai
import os

from dotenv import find_dotenv, load_dotenv

from langsmith import traceable


_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

APP_VERSION = os.environ["APP_VERSION"]
MODEL_NAME = os.environ["MODEL_NAME"]
MODEL_PROVIDER = os.environ["MODEL_PROVIDER"]

In [4]:
import nest_asyncio

from langsmith.wrappers import wrap_openai
from utils import get_vector_db_retriever


RAG_SYSTEM_PROMPT = """
    You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the latest question in the conversation. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
"""

openai_client = wrap_openai(openai.Client())
nest_asyncio.apply()
retriever = get_vector_db_retriever()


@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)


@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )


@traceable(run_type="chain")
def langsmith_rag_with_wrap_openai(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

In [5]:
question = "How do I trace with wrap_openai?"
ai_answer = langsmith_rag_with_wrap_openai(question)
ai_answer

'To trace with `wrap_openai`, first import it using `from langsmith.wrappers import wrap_openai`, then wrap your OpenAI client with `openai_client = wrap_openai(OpenAI())`. Make sure to set the environment variable `LANGSMITH_TRACING` to \'true\' to enable logging, and also set your `LANGSMITH_API_KEY`. By default, traces will be logged to a project named "default," but you can log them to a different project if needed.'

In [6]:
# the wrapped OpenAI client accepts all the same langsmith_extra parameters as @traceable decorated functions.

messages = [
    {
        "role": "user",
        "content": "What color is the sky?"
    }
]

openai_client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages,
    langsmith_extra={"metadata": {"foo": "bar"}},
)

ChatCompletion(id='chatcmpl-BIEHqtm99B5e7cXBVq62y03xXXKJa', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The color of the sky typically appears blue during the day due to the scattering of sunlight by the atmosphere. This phenomenon, known as Rayleigh scattering, causes shorter blue wavelengths of light to scatter more than other colors. At sunrise and sunset, the sky can display a range of colors, including orange, pink, and red, as the light passes through more of the atmosphere. Additionally, the sky can appear gray or even white on overcast days.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1743684522, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_b376dfbbd5', usage=CompletionUsage(completion_tokens=91, prompt_tokens=13, total_tokens=104, completion_tokens_details=CompletionTokensDetails(accepted_predicti